In [18]:
from manim import *
import random


config.media_width = "75%"
config.verbosity = "WARNING"

In [19]:


class NeutronScatteringWithDistribution(ThreeDScene):
    def construct(self):
        # Set up 3D axes
        axes = ThreeDAxes(
            x_range=[-5, 5, 1],
            y_range=[-5, 5, 1],
            z_range=[-5, 5, 1]
        )
        labels = axes.get_axis_labels(
        Text("x-axis").scale(0.7), Text("y-axis").scale(0.45), Text("z-axis").scale(0.45)
        )

        cube = Prism(dimensions=[1.,1,0.2])

        
        # Add axes to the scene
        self.add(axes, cube)
        
        # Set camera angle for better 3D perspective
        self.set_camera_orientation(phi=75 * DEGREES, theta=30 * DEGREES)
        self.begin_ambient_camera_rotation(rate=0.2)

        # Function to animate a neutron scattering
        def animate_neutron(color, phi, theta):
            # Start the neutron at (0, 0, -1)
            neutron = Sphere(center=[0, 0, -1], radius=0.1, color=color)
            self.add(neutron)

            # Move neutron to origin (scattering point)
            neutron.generate_target()
            neutron.target.move_to([0, 0, 0])

            # Calculate final position after scattering based on phi and theta
            r = 1.5  # Final distance from origin after scattering
            x_final = r * np.sin(phi) * np.cos(theta)
            y_final = r * np.sin(phi) * np.sin(theta)
            z_final = r * np.cos(phi)
            
            # Animate the neutron to the origin, then to its final scattered position
            self.play(MoveToTarget(neutron), run_time=1)
            self.play(neutron.animate.move_to([x_final, y_final, z_final]), run_time=1)

        # Animate the first neutron scattering
        animate_neutron(color=BLUE, phi=10 * DEGREES, theta=10 * DEGREES)
        
        # Animate the second neutron scattering
        animate_neutron(color=RED, phi=30 * DEGREES, theta=-20 * DEGREES)

        # Show multiple scatterings
        scatter_colors = [YELLOW, GREEN, ORANGE, PINK, YELLOW_A, GREEN_A]
        scatter_angles = [(20, 40), (25, -30), (12, 15), (13, -10), (3, -76), (11, 80)]
        
        for i, (phi_deg, theta_deg) in enumerate(scatter_angles):
            phi = phi_deg * DEGREES
            theta = theta_deg * DEGREES
            animate_neutron(color=scatter_colors[i], phi=phi, theta=theta)
        
        # Create a cone to represent the azimuthal distribution
        cone = Cone(
            direction=[0, 0, -1],     # Pointing along the positive z-axis
            height=1.5,                # Height of the cone
            base_radius=1.5,           # Radius of the cone's base
            color=GREY,
        )
            
        
        # Display the cone in the scene to show the azimuthal distribution
        self.play(FadeIn(cone))
        
        # Ambient camera rotation to visualize the cone and scatter points
        self.begin_ambient_camera_rotation(rate=0.2)
        self.wait(3)


# don't remove below command for run button to work
%manim -qm -v WARNING NeutronScatteringWithDistribution

Manim Community v0.18.1

In [20]:
from manim import *

class NeutronScattering2DView(Scene):
    def construct(self):
        # Draw a circle to represent the azimuthal scattering plane
        circle = Circle(radius=2, color=GREY)
        circle_label = Text("Azimuthal Distribution").scale(0.5).next_to(circle, UP)
        self.play(Create(circle), Write(circle_label))

        # Add a central point to represent the z-axis
        center_point = Dot(point=[0, 0, 0], color=WHITE, radius=0.08)
        center_label = Text("z-axis").scale(0.4).next_to(center_point, DOWN)
        self.play(FadeIn(center_point), Write(center_label))

        # Function to add neutron scatter points inside the circle
        def plot_scatter_point(color, r, theta):
            # Calculate 2D position based on random radius and angle
            x_final = r * np.cos(theta)
            y_final = r * np.sin(theta)

            # Create a dot at the scatter position
            dot = Dot(point=[x_final, y_final, 0], color=color, radius=0.05)
            self.play(FadeIn(dot), run_time=0.1)

        # Generate multiple scatter points within the full circle
        num_points = 10  # Number of scatter points
        colors = [BLUE, RED, YELLOW, GREEN, ORANGE, PINK, PURPLE, TEAL, MAROON, GOLD]
        
        for _ in range(num_points):
            # Randomize radial distance (0 to 2) and angle (0 to 2π) for each point
            r = random.uniform(0, 2)  # Distance from center, within circle radius
            theta = random.uniform(0, 2 * PI)  # Angle around the circle
            color = random.choice(colors)
            plot_scatter_point(color=color, r=r, theta=theta)

        self.wait(2)


%manim -qm -v WARNING NeutronScattering2DView

Manim Community v0.18.1

In [21]:
%manim -qm -v WARNING NeutronScattering2DWithHistogram

class NeutronScattering2DWithPolarizedHistogram(ThreeDScene):
    def construct(self):
        # Create the circle on the left side without a grid
        circle = Circle(radius=3, color=WHITE).shift(LEFT * 3.5)
        center_dot = Dot(circle.get_center(), color=WHITE)  # Center point representing the z-axis

        # Adding the circle and center point
        self.add(circle, center_dot)

        # Generate points distributed inside the circle with polarization (more on one side)
        np.random.seed(10)
        angles = np.random.normal(loc=150, scale=40, size=200) % 360
        radii = np.random.uniform(0, 3, size=200)  # Random radii to place points inside the circle
        points = [circle.get_center() + np.array([
            r * np.cos(angle * DEGREES),
            r * np.sin(angle * DEGREES),
            0
        ]) for angle, r in zip(angles, radii)]
        
        # Create dots for scatter points
        dots = VGroup(*[Dot(point, color=BLUE).scale(0.5) for point in points])

        # Display all scatter points inside the circle
        self.play(FadeIn(dots), run_time=2)

        # Set up histogram data
        bin_width = 5
        bins = np.arange(0, 360 + bin_width, bin_width)
        counts, _ = np.histogram(angles, bins=bins)

        # Create axes for the histogram on the right
        hist_plane = Axes(
            x_range=[0, 360, 45],  # Bins cover 0 to 360 in steps of 45 degrees
            y_range=[0, max(counts) + 1, 1],  # Counts axis slightly above max count
            x_length=5,
            y_length=3,
            axis_config={"color": WHITE}
        ).shift(RIGHT * 3.5)

        # Add bin segments on x-axis
        bin_lines = VGroup(*[
            Line(
                start=hist_plane.c2p(bin, 0),
                end=hist_plane.c2p(bin, max(counts) + 1),
                color=GRAY
            ) for bin in bins
        ])

        # Add the histogram axis and bin segments
        self.play(Create(hist_plane), Create(bin_lines))

        # Plot individual points on the histogram for each bin
        hist_dots = VGroup()
        for i, (count, bin) in enumerate(zip(counts, bins[:-1])):
            bin_center = bin + bin_width / 2
            for j in range(count):
                dot = Dot(hist_plane.c2p(bin_center, j + 1), color=RED).scale(0.3)
                hist_dots.add(dot)
                # Show each dot incrementally
                self.play(FadeIn(dot), run_time=0.05)
        
        # Show the completed histogram with all dots
        self.wait(2)

        # Create a 2D plane (circle with scatter points)
        plane = NumberPlane(
            x_range=[-5, 5, 1],
            y_range=[-5, 5, 1],
            background_line_style={"stroke_opacity": 0.3}
        )
        circle = Circle(radius=3, color=WHITE)
        center_dot = Dot(ORIGIN, color=WHITE)  # Center point representing z-axis
        
        # Adding the plane, circle, and center point
        self.add(plane, circle, center_dot)

        # Generate polarized scatter points with more on one side of the circle
        np.random.seed(10)
        angles = np.random.normal(loc=150, scale=40, size=200) % 360
        points = [circle.point_at_angle(angle * DEGREES) for angle in angles]
        dots = VGroup(*[Dot(point, color=BLUE).scale(0.5) for point in points])

        # Display all scatter points
        self.play(FadeIn(dots), run_time=2)

        # Initialize histogram data
        bin_width = 5
        bins = np.arange(0, 360 + bin_width, bin_width)
        counts, _ = np.histogram(angles, bins=bins)

        # Create the histogram to the side
        hist_plane = Axes(
            x_range=[0, 360, 45],  # Bins cover 0 to 360 in steps of 45 degrees
            y_range=[0, max(counts) + 1, 1],  # Counts axis slightly above max count
            x_length=5,
            y_length=3,
            axis_config={"color": WHITE}
        ).shift(RIGHT * 5)

        # Create histogram bars
        bars = VGroup(
            *[
                Rectangle(
                    width=(bin_width / 360) * hist_plane.x_length,
                    height=(count / max(counts)) * hist_plane.y_length,
                    fill_color=BLUE,
                    fill_opacity=0.7,
                ).next_to(hist_plane.c2p(bin + bin_width / 2, 0), UP, buff=0)
                for count, bin in zip(counts, bins[:-1])
            ]
        )

        # Display the histogram bars incrementally as scatter points are added
        self.play(Create(hist_plane), *[GrowFromEdge(bar, DOWN) for bar in bars])

        # Show gradual updates to the histogram by animating bars
        for i, bar in enumerate(bars):
            self.play(FadeIn(bar), run_time=0.1)
            self.wait(0.05)

        # Final pause to observe the completed histogram
        self.wait(2)




Manim Community v0.18.1

In [22]:
from manim import *
import numpy as np
import random

class NeutronScattering2DWithPlot(Scene):
    def construct(self):
        # ----------------------------
        # Part 1: Set Up the Circle
        # ----------------------------

        # Draw a circle on the left side to represent the azimuthal scattering plane
        circle = Circle(radius=2, color=GREY).shift(LEFT * 3)
        self.play(Create(circle))

        # Add a central point to represent the z-axis
        center_point = Dot(point=[-3, 0, 0], color=WHITE, radius=0.08)
        self.play(FadeIn(center_point))

        # ----------------------------
        # Part 2: Set Up the Plot for Histogram
        # ----------------------------

        # Define bins and axes for the scatter angle plot on the right side
        bin_width = 5
        bins = np.arange(0, 360 + bin_width, bin_width)

        # Create axes for the scatter angle plot on the right
        axes = Axes(
            x_range=[0, 360, 45],       # Angles from 0 to 360 in steps of 45
            y_range=[0, 10, 1],         # Initial count axis, adjust dynamically
            x_length=6,
            y_length=3,
            axis_config={"color": WHITE}
        ).shift(RIGHT * 3)

        # Create x and y labels without using LaTeX
        x_label = Text("Angle (Degrees)").next_to(axes, DOWN)
        y_label = Text("Counts").next_to(axes, LEFT)
        self.play(Create(axes), FadeIn(x_label), FadeIn(y_label))

        # Initialize list to hold markers for the plot points
        self.plot_points = VGroup()

        # ----------------------------
        # Part 3: Define Scatter Point Addition
        # ----------------------------

        # Initialize list to keep track of scatter point angles
        scatter_points = []

        # Define a function to add scatter points and update the plot with markers
        def add_scatter_point_and_update_plot(color, r, theta):
            # Calculate position within the circle
            x_final = r * np.cos(theta) - 3  # Adjust for circle's left position
            y_final = r * np.sin(theta)

            # Create and display the scatter point
            dot = Dot(point=[x_final, y_final, 0], color=color, radius=0.05)
            self.play(FadeIn(dot), run_time=0.2)

            # Convert angle to degrees and add to scatter_points list
            angle_deg = np.degrees(theta) % 360
            scatter_points.append(angle_deg)

            # Remove existing plot points
            if len(self.plot_points) > 0:
                self.remove(*self.plot_points)
                self.plot_points = VGroup()  # Reset plot_points to an empty VGroup

            # Recompute histogram counts based on updated scatter_points
            counts, _ = np.histogram(scatter_points, bins=bins)
            bin_centers = 0.5 * (bins[1:] + bins[:-1])

            # Dynamically adjust y_range if counts exceed current maximum
            max_count = max(counts)
            if max_count + 1 > axes.y_range[1]:
                axes.y_range = [0, max_count + 1, 1]
                axes.update_range()

            # Create new plot markers only for bins with counts > 0
            for i, count in enumerate(counts):
                if count > 0:  # Only plot non-zero counts
                    x_pos = bin_centers[i]
                    marker = Dot(
                        point=axes.c2p(x_pos, count),
                        color=BLUE,
                        radius=0.08
                    )
                    self.plot_points.add(marker)

            # Animate the new plot markers appearing
            self.play(FadeIn(self.plot_points), run_time=0.2)

        # ----------------------------
        # Part 4: Generate Scatter Points Dynamically
        # ----------------------------

        # Define number of scatter points and available colors
        num_points = 20
        colors = [BLUE, RED, YELLOW, GREEN, ORANGE, PINK, PURPLE, TEAL, MAROON, GOLD]

        # Generate scatter points one by one, updating the plot each time
        for _ in range(num_points):
            r = random.uniform(0, 2)          # Random radial distance within circle
            theta = random.uniform(0, 2 * np.pi)  # Random angle in radians
            color = random.choice(colors)
            add_scatter_point_and_update_plot(color=color, r=r, theta=theta)
            self.wait(0.1)  # Brief pause between adding points

        # Final pause to observe the completed animation
        self.wait(2)


%manim -qm -v WARNING NeutronScattering2DWithPlot



Manim Community v0.18.1

In [23]:
from manim import *
import numpy as np
import random

class NeutronScatteringSeparatePlots(Scene):
    def construct(self):
        # ----------------------------
        # Part 1: Set Up the Circle
        # ----------------------------
        
        # Draw a circle to represent the azimuthal scattering plane
        circle = Circle(radius=2, color=GREY).shift(LEFT * 3)
        self.play(Create(circle))

        # Add a central point to represent the z-axis
        center_point = Dot(point=[-3, 0, 0], color=WHITE, radius=0.08)
        self.play(FadeIn(center_point))

        # ----------------------------
        # Part 2: Set Up the Plot for Hit Counts
        # ----------------------------
        
        # Define bins and axes for the hit count plot
        bin_width = 5
        bins = np.arange(0, 360 + bin_width, bin_width)

        # Create axes for the hit count plot
        axes = Axes(
            x_range=[0, 360, 45],       # Angles from 0 to 360 in steps of 45
            y_range=[0, 10, 2],         # Counts axis, ranging from 0 to 10
            x_length=6,
            y_length=3,
            axis_config={"color": WHITE}
        ).shift(RIGHT * 3)

        # Create x and y labels
        x_label = Text("Angle (Degrees)").next_to(axes, DOWN)
        y_label = Text("Counts").next_to(axes, LEFT)
        self.play(Create(axes), FadeIn(x_label), FadeIn(y_label))

        # Initialize groups to hold plot markers for blue and red points
        blue_points = VGroup()
        red_points = VGroup()

        # ----------------------------
        # Part 3: Define Distributions and Updates
        # ----------------------------

        # Initialize lists to keep track of scatter point angles for left and right groups
        left_angles = []
        right_angles = []

        # Function to add scatter points for left or right group
        def add_scatter_point(color, r, theta, group):
            # Calculate position within the circle
            x_final = r * np.cos(theta) - 3  # Adjust for circle's left position
            y_final = r * np.sin(theta)

            # Create and display the scatter point
            dot = Dot(point=[x_final, y_final, 0], color=color, radius=0.05)
            self.play(FadeIn(dot), run_time=0.1)

            # Convert angle to degrees and add to the corresponding group
            angle_deg = np.degrees(theta) % 360
            group.append(angle_deg)

        # Function to update the plot showing the counts
        def update_hit_plot():
            # Remove existing points
            self.remove(*blue_points, *red_points)
            blue_points.submobjects = []
            red_points.submobjects = []

            # Compute histogram counts for left and right groups
            left_counts, _ = np.histogram(left_angles, bins=bins)
            right_counts, _ = np.histogram(right_angles, bins=bins)
            bin_centers = 0.5 * (bins[1:] + bins[:-1])

            # Create new plot points for blue and red distributions
            for i, (blue_count, red_count) in enumerate(zip(left_counts, right_counts)):
                x_pos = bin_centers[i]
                if blue_count > 0:
                    blue_point = Dot(
                        point=axes.c2p(x_pos, blue_count),
                        color=BLUE,
                        radius=0.08
                    )
                    blue_points.add(blue_point)
                if red_count > 0:
                    red_point = Dot(
                        point=axes.c2p(x_pos, red_count),
                        color=RED,
                        radius=0.08
                    )
                    red_points.add(red_point)

            # Animate the new points appearing
            self.play(FadeIn(blue_points, red_points), run_time=0.2)

        # ----------------------------
        # Part 4: Generate Scatter Points
        # ----------------------------
        
        # Number of scatter points for each group
        num_points = 100

        # Generate scatter points for left (blue) and right (red) distributions
        for _ in range(num_points):
            # Generate a point for the left distribution (blue)
            r = random.uniform(0, 2)
            theta = random.uniform(0, 2 * np.pi)
            if random.random() < 0.6:  # 60% chance for blue
                add_scatter_point(color=BLUE, r=r, theta=theta, group=left_angles)

            # Generate a point for the right distribution (red)
            r = random.uniform(0, 2)
            theta = random.uniform(0, 2 * np.pi)
            if random.random() < 0.6:  # 60% chance for red
                add_scatter_point(color=RED, r=r, theta=theta, group=right_angles)

            # Update the hit count plot after each pair of points
            update_hit_plot()

        # Final pause to observe the completed animation
        self.wait(2)



%manim -qm -v WARNING NeutronScatteringSeparatePlots


Manim Community v0.18.1

In [24]:
from manim import *

class NeutronFinalAsymmetryWithFit(Scene):
    def construct(self):
        # ----------------------------
        # Part 1: Set Up the Plot
        # ----------------------------

        # Define bins and axes for the hit count plot
        bin_width = 30
        bins = np.arange(-180, 180 + bin_width, bin_width)

        # Create axes for the hit count plot
        axes = Axes(
            x_range=[-180, 180, 45],  # Angles from -180 to 180 in steps of 45
            y_range=[0, 100, 10],     # Counts axis, ranging from 0 to 100
            x_length=8,
            y_length=4,
            axis_config={"color": WHITE}
        ).shift(UP)

        # Create x and y labels
        x_label = Text("Angle (Degrees)").next_to(axes, DOWN)
        y_label = Text("Counts").next_to(axes, LEFT)
        self.play(Create(axes), FadeIn(x_label), FadeIn(y_label))

        # Initialize groups to hold plot markers and error bars
        blue_points = VGroup()
        red_points = VGroup()
        blue_error_bars = VGroup()
        red_error_bars = VGroup()

        # Initialize lists to keep track of scatter point angles for left and right groups
        left_angles = []
        right_angles = []

        # ----------------------------
        # Part 2: Generate Scatter Points with Artificial Asymmetry
        # ----------------------------

        # Number of scatter points for each group
        num_points = 300

        # Generate scatter points for left (blue) and right (red) distributions
        for _ in range(num_points):
            # Generate a point for the left distribution (blue) with weaker bias
            theta = random.uniform(-np.pi, np.pi)
            bias_factor = 1.2 + 1.0 * np.cos(theta)  # Reduced bias
            if random.random() < bias_factor:
                left_angles.append(np.degrees(theta))

            # Generate a point for the right distribution (red) with complementary bias
            theta = random.uniform(-np.pi, np.pi)
            bias_factor = 1.2 - 1.0 * np.cos(theta)  # Reduced bias
            if random.random() < bias_factor:
                right_angles.append(np.degrees(theta))

        # Compute histogram counts for left and right groups
        left_counts, _ = np.histogram(left_angles, bins=bins)
        right_counts, _ = np.histogram(right_angles, bins=bins)
        bin_centers = 0.5 * (bins[1:] + bins[:-1])

        # Create plot points and error bars for blue and red distributions
        for i, (blue_count, red_count) in enumerate(zip(left_counts, right_counts)):
            x_pos = bin_centers[i]

            # Add blue points and error bars
            if blue_count > 0:
                blue_point = Dot(
                    point=axes.c2p(x_pos, blue_count),
                    color=BLUE,
                    radius=0.08
                )
                blue_points.add(blue_point)

                # Add vertical error bar
                error = np.sqrt(blue_count)
                vertical_error_bar = Line(
                    start=axes.c2p(x_pos, blue_count - error),
                    end=axes.c2p(x_pos, blue_count + error),
                    color=BLUE
                )
                blue_error_bars.add(vertical_error_bar)

                # Add horizontal error bar (x error)
                horizontal_error_bar = Line(
                    start=axes.c2p(x_pos - bin_width / 2, blue_count),
                    end=axes.c2p(x_pos + bin_width / 2, blue_count),
                    color=BLUE
                )
                blue_error_bars.add(horizontal_error_bar)

            # Add red points and error bars
            if red_count > 0:
                red_point = Dot(
                    point=axes.c2p(x_pos, red_count),
                    color=RED,
                    radius=0.08
                )
                red_points.add(red_point)

                # Add vertical error bar
                error = np.sqrt(red_count)
                vertical_error_bar = Line(
                    start=axes.c2p(x_pos, red_count - error),
                    end=axes.c2p(x_pos, red_count + error),
                    color=RED
                )
                red_error_bars.add(vertical_error_bar)

                # Add horizontal error bar (x error)
                horizontal_error_bar = Line(
                    start=axes.c2p(x_pos - bin_width / 2, red_count),
                    end=axes.c2p(x_pos + bin_width / 2, red_count),
                    color=RED
                )
                red_error_bars.add(horizontal_error_bar)

        # Animate the appearance of the hit count plot with error bars
        self.play(FadeIn(blue_points, red_points), run_time=0.5)
        self.play(Create(blue_error_bars), Create(red_error_bars))  # Hit count plot
        self.wait(3)

        # ----------------------------
        # Part 3: Transition to Asymmetry Plot
        # ----------------------------

        # Compute the asymmetry and errors
        asymmetry = []
        asymmetry_errors = []
        for blue_count, red_count in zip(left_counts, right_counts):
            if blue_count + red_count > 0:  # Avoid division by zero
                asymmetry_value = (blue_count - red_count) / (blue_count + red_count)
                asymmetry_error = (
                    2 * np.sqrt((blue_count * red_count)) / (blue_count + red_count)
                )
            else:
                asymmetry_value = 0  # Default for bins with no counts
                asymmetry_error = 0

            asymmetry.append(asymmetry_value)
            asymmetry_errors.append(asymmetry_error)

        # Create axes for the asymmetry plot
        asymmetry_axes = Axes(
            x_range=[-180, 180, 45],
            y_range=[-5, 5, 1],  # Asymmetry ranges from -1.2 to 1.2
            x_length=8,
            y_length=4,
            axis_config={"color": WHITE}
        ).shift(UP)

        asymmetry_y_label = Text("A").next_to(asymmetry_axes, LEFT)

        # Prepare asymmetry points and error bars
        asymmetry_points = VGroup()
        asymmetry_error_bars = VGroup()

        print(asymmetry_errors)

        for i, (value, error) in enumerate(zip(asymmetry, asymmetry_errors)):
            x_pos = bin_centers[i]

            # Add asymmetry point
            asymmetry_point = Dot(
                point=asymmetry_axes.c2p(x_pos, value),
                color=YELLOW,
                radius=0.08
            )
            asymmetry_points.add(asymmetry_point)

            # Add vertical error bar (y-error)
            vertical_error_bar = Line(
                start=asymmetry_axes.c2p(x_pos, value - (error)),
                end=asymmetry_axes.c2p(x_pos, value + (error)),
                color=YELLOW
            )
            asymmetry_error_bars.add(vertical_error_bar)

            # Add horizontal error bar (x-error for bin width)
            horizontal_error_bar = Line(
                start=asymmetry_axes.c2p(x_pos - bin_width / 2, value),
                end=asymmetry_axes.c2p(x_pos + bin_width / 2, value),
                color=YELLOW
            )
            asymmetry_error_bars.add(horizontal_error_bar)


        # Transition from the hit count plot to the asymmetry plot
        self.play(Transform(axes, asymmetry_axes), Transform(y_label, asymmetry_y_label))
        self.play(FadeOut(blue_points, red_points, blue_error_bars, red_error_bars))
        self.play(FadeIn(asymmetry_points), FadeIn(asymmetry_error_bars))  # Asymmetry plot

        # ----------------------------
        # Part 4: Overlay Cosine Fit
        # ----------------------------

        # Convert bin centers from degrees to radians for the fit
        bin_centers_radians = bin_centers * np.pi / 180

        # Estimate amplitude from the observed asymmetry
        amplitude = max(abs(np.array(asymmetry)))  # Dynamically set amplitude

        # Define a cosine function for the fit
        fit_curve = FunctionGraph(
            lambda x: amplitude * np.cos(x),  # x here is already in radians
            x_range=[-PI, PI],
            color=GREEN
        ).move_to(asymmetry_axes.c2p(0, 0))



        # Overlay the fit
        self.play(Create(fit_curve), run_time=2)

        # Final pause to observe the asymmetry plot with fit
        self.wait(2)


%manim -qm -v WARNING NeutronFinalAsymmetryWithFit


Manim Community v0.18.1

[0.7453559924999299, 0.7355970484510567, 0.9967948635501689, 0.989743318610787, 0.9428090415820632, 0.7332121111929344, 0.8558395617924729, 0.8588691152028785, 0.999260081289737, 0.989743318610787, 0.7261843774138906, 0.9315409787235999]


In [25]:
%%manim -qm NucleonScattering

from manim import *

class NucleonScattering(ThreeDScene):
    def construct(self):
        # Set up the camera angle
        self.set_camera_orientation(phi=75 * DEGREES, theta=30 * DEGREES)
        
        # Create the steel analyzer (3D prism)
        analyzer = Prism(
            dimensions=[0.5, 2, 2],  # width, height, depth
            fill_color=GREY,
            fill_opacity=0.7,
            stroke_width=1
        )
        
        # Create the nucleon (sphere)
        nucleon = Sphere(
            radius=0.1,
            fill_color=BLUE,
            fill_opacity=0.8
        ).set_color(BLUE)
        
        # Starting position for nucleon
        nucleon.move_to([-3, 0, 0])
        
        # Add objects to scene
        self.add(analyzer)
        self.add(nucleon)
        
        # Define the path for scattering
        # First part: straight line towards analyzer
        path1 = Line([-3, 0, 0], [0, 0, 0])
        # Second part: scattered path (small angle)
        path2 = Line([0, 0, 0], [2, 0.5, 0])
        
        # Create the animation
        # First movement: approach
        self.play(
            MoveAlongPath(nucleon, path1),
            rate_func=linear,
            run_time=2
        )
        

        # Second movement: scattering
        self.play(
            MoveAlongPath(nucleon, path2),
            rate_func=linear,
            run_time=1.5
        )
        
        # Optional: show trajectory
        trajectory = VGroup(
            DashedLine([-3, 0, 0], [0, 0, 0]),
            DashedLine([0, 0, 0], [2, 0.5, 0])
        ).set_color(GREY_C)
        
        self.play(
            Create(trajectory),
            run_time=1
        )
        
        # Hold final frame
        self.wait()

Manim Community v0.18.1

In [26]:
%%manim -qm NucleonScattering

from manim import *

class NucleonScattering(ThreeDScene):
    def construct(self):
        # Set up the camera angle
        self.set_camera_orientation(phi=75 * DEGREES, theta=-50 * DEGREES)
        
        # Begin ambient rotation
        self.begin_ambient_camera_rotation(rate=0.1)
        
        # Create the steel analyzer (3D prism)
        analyzer = Prism(
            dimensions=[0.5, 2, 2],  # width, height, depth
            fill_color=GREY,
            fill_opacity=0.3,
            stroke_width=1
        )
        
        # Create the nucleon (sphere)
        nucleon = Sphere(
            radius=0.1,
            fill_color=BLUE,
            fill_opacity=0.8
        ).set_color(BLUE)
        
        # Starting position for nucleon
        nucleon.move_to([-3, 0, 0])
        
        # Add objects to scene
        self.add(analyzer)
        self.add(nucleon)
        
        # Define the paths for movement
        path1 = Line([-3, 0, 0], [0, 0, 0])
        path2 = Line([0, 0, 0], [2, 0.5, 0])
        
        # First movement: approach
        self.play(
            MoveAlongPath(nucleon, path1),
            rate_func=linear,
            run_time=2
        )
        
        # Add a small pause at collision
        self.wait(0.2)
        
        # Second movement: scattering
        self.play(
            MoveAlongPath(nucleon, path2),
            rate_func=linear,
            run_time=1.5
        )
        
        # Draw the complete trajectory
        trajectory = VGroup(
            DashedLine([-3, 0, 0], [0, 0, 0]),
            DashedLine([0, 0, 0], [2, 0.5, 0])
        ).set_color(GREY_C)
        
        self.play(
            Create(trajectory),
            run_time=1
        )
        
        # Create and draw the angle arc
        # First create a reference line along x-axis
        ref_line = DashedLine([0, 0, 0], [2, 0, 0], color=RED_A)
        angle = Angle(
            Line([0, 0, 0], [2, 0, 0]),  # Reference line
            Line([0, 0, 0], [2, 0.5, 0]),  # Scattered path
            radius=0.5,
            color=YELLOW,
            stroke_width=2
        )
        
        # Add angle label
        angle_value = 14  # approximate angle in degrees
        angle_label = MathTex(f"{angle_value}°").scale(0.5)
        angle_label.next_to(angle, RIGHT)
        
        # Show angle visualization
        self.play(
            Create(ref_line),
            Create(angle),
            Write(angle_label),
            run_time=1
        )
        

        
        # Hold final frame
        self.wait(3)

        # Stop the rotation before ending
        self.stop_ambient_camera_rotation()

Manim Community v0.18.1

In [27]:
%%manim -qm NucleonScattering

from manim import *

class NucleonScattering(ThreeDScene):
    def construct(self):
        # Set up the camera angle
        self.set_camera_orientation(phi=75 * DEGREES, theta=-50 * DEGREES)
        
        # Begin ambient rotation
        self.begin_ambient_camera_rotation(rate=0.1)
        
        # Create the steel analyzer (3D prism)
        analyzer = Prism(
            dimensions=[0.5, 2, 2],  # width, height, depth
            fill_color=GREY,
            fill_opacity=0.3,
            stroke_width=1
        )
        
        # Create the nucleons (spheres)
        nucleon1 = Sphere(
            radius=0.1,
            fill_color=BLUE,
            fill_opacity=0.8
        ).set_color(BLUE)
        
        nucleon2 = Sphere(
            radius=0.1,
            fill_color=BLUE,
            fill_opacity=0.8
        ).set_color(BLUE)
        
        # Starting positions
        nucleon1.move_to([-3, 0, 0])
        nucleon2.move_to([-3, 0.5, 0])
        
        # Add objects to scene
        self.add(analyzer)
        self.add(nucleon1)
        
        # Define the paths for both nucleons
        path1_1 = Line([-3, 0, 0], [0, 0, 0])
        path1_2 = Line([0, 0, 0], [2, 0.5, 0])
        
        path2_1 = Line([-3, 0.5, 0], [0, 0.5, 0])
        path2_2 = Line([0, 0.5, 0], [2, -0.5, 0])
        
        # First nucleon scattering
        self.play(
            MoveAlongPath(nucleon1, path1_1),
            rate_func=linear,
            run_time=2
        )
        
        self.wait(0.2)
        
        self.play(
            MoveAlongPath(nucleon1, path1_2),
            rate_func=linear,
            run_time=1.5
        )
        
        # Draw first trajectory
        trajectory1 = VGroup(
            DashedLine([-3, 0, 0], [0, 0, 0]),
            DashedLine([0, 0, 0], [2, 0.5, 0])
        ).set_color(GREY_C)
        
        self.play(
            Create(trajectory1),
            run_time=1
        )
        
        # Show first angle measurement
        ref_line1 = DashedLine([0, 0, 0], [2, 0, 0], color=RED_A)
        angle1 = Angle(
            Line([0, 0, 0], [2, 0, 0]),  # Reference line
            Line([0, 0, 0], [2, 0.5, 0]),  # Scattered path
            radius=0.5,
            color=YELLOW,
            stroke_width=2
        )
        
        angle_label1 = MathTex(r"14^{\circ}").scale(2)
        angle_label1.next_to(angle1, RIGHT)
        
        self.play(
            Create(ref_line1),
            Create(angle1),
            Write(angle_label1),
            run_time=1
        )
        
        # Wait briefly to show first angle
        self.wait(1)
        
        # Remove first nucleon and its measurements
        self.play(
            FadeOut(nucleon1),
            FadeOut(ref_line1),
            FadeOut(angle1),
            FadeOut(angle_label1),
            FadeOut(trajectory1)
        )
        
        # Add second nucleon
        self.add(nucleon2)
        
        # Second nucleon scattering
        self.play(
            MoveAlongPath(nucleon2, path2_1),
            rate_func=linear,
            run_time=2
        )
        
        self.wait(0.2)
        
        self.play(
            MoveAlongPath(nucleon2, path2_2),
            rate_func=linear,
            run_time=1.5
        )
        
        # Draw second trajectory
        trajectory2 = VGroup(
            DashedLine([-3, 0.5, 0], [0, 0.5, 0]),
            DashedLine([0, 0.5, 0], [2, -0.5, 0])
        ).set_color(GREY_C)
        
        self.play(
            Create(trajectory2),
            run_time=1
        )
        
        # Show second angle measurement - corrected to show acute angle
        ref_line2 = DashedLine([0, 0.5, 0], [2, 0.5, 0], color=RED_A)
        angle2 = Angle(
            Line([0, 0.5, 0], [2, 0.5, 0]),  # Reference line
            Line([0, 0.5, 0], [2, -0.5, 0]),  # Scattered path
            radius=0.5,
            color=YELLOW,
            stroke_width=2,
            other_angle=True  # This ensures we get the acute angle
        )
        
        angle_label2 = MathTex(r"-27^{\circ}").scale(3)
        angle_label2.next_to(angle2, UP)
        
        self.play(
            Create(ref_line2),
            Create(angle2),
            Write(angle_label2),
            run_time=1
        )
        
        # Stop the rotation before ending
        self.stop_ambient_camera_rotation()
        
        # Hold final frame
        self.wait()

Manim Community v0.18.1

In [28]:
%%manim -qm NucleonScattering

from manim import *
import numpy as np

class NucleonScattering(ThreeDScene):
    def construct(self):
        # Set up the camera angle
        self.set_camera_orientation(phi=75 * DEGREES, theta=-50 * DEGREES)
        
        # Begin ambient rotation
        self.begin_ambient_camera_rotation(rate=0.2)
        
        # Create the steel analyzer (3D prism)
        analyzer = Prism(
            dimensions=[0.5, 2, 2],
            fill_color=GREY,
            fill_opacity=0.3,
            stroke_width=1
        )
        
        # Define different scattering angles (in degrees)
        scatter_angles = [14, 27, -20, 18, -25, 22]
        run_times = [1.5, 1.2, 0.8, 0.5, 0.4, 0.3]  # Decreasing times for each event
        
        # Function to create a scattering event
        def create_scattering_event(start_y, angle_deg, run_time_factor):
            nucleon = Sphere(radius=0.1, fill_opacity=0.8).set_color(BLUE)
            nucleon.move_to([-3, start_y, 0])
            
            # Calculate end point based on angle
            angle_rad = angle_deg * DEGREES
            end_x = 2
            end_y = start_y + np.tan(angle_rad) * 2
            
            path1 = Line([-3, start_y, 0], [0, start_y, 0])
            path2 = Line([0, start_y, 0], [end_x, end_y, 0])
            
            # Add nucleon
            self.add(nucleon)
            
            # Animate movement
            self.play(
                MoveAlongPath(nucleon, path1),
                rate_func=linear,
                run_time=2 * run_time_factor
            )
            
            self.wait(0.1 * run_time_factor)
            
            self.play(
                MoveAlongPath(nucleon, path2),
                rate_func=linear,
                run_time=1.5 * run_time_factor
            )
            
            # Draw trajectory
            trajectory = VGroup(
                DashedLine([-3, start_y, 0], [0, start_y, 0]),
                DashedLine([0, start_y, 0], [end_x, end_y, 0])
            ).set_color(GREY_C)
            
            self.play(Create(trajectory), run_time=0.5 * run_time_factor)
            
            # Show angle
            ref_line = DashedLine([0, start_y, 0], [2, start_y, 0], color=RED_A)

            # Create angle visualization
            if angle_deg > 0:
                angle_arc = Angle(
                    Line([0, start_y, 0], [2, start_y, 0]),
                    Line([0, start_y, 0], [end_x, end_y, 0]),
                    radius=0.5,
                    color=YELLOW,
                    stroke_width=2,
                )
            else:
                angle_arc = Angle(
                    Line([0, start_y, 0], [2, start_y, 0]),
                    Line([0, start_y, 0], [end_x, end_y, 0]),
                    radius=0.5,
                    color=YELLOW,
                    stroke_width=2,
                    other_angle=True
                )
            
            angle_label = MathTex(f"{abs(angle_deg)}^{{\circ}}").scale(0.5)
            angle_label.next_to(angle_arc, RIGHT)
            
            self.play(
                Create(ref_line),
                Create(angle_arc),
                Write(angle_label),
                run_time=0.5 * run_time_factor
            )
            
            self.wait(0.3 * run_time_factor)
            
            # Remove everything
            self.play(
                FadeOut(nucleon),
                FadeOut(ref_line),
                FadeOut(angle_arc),
                FadeOut(angle_label),
                FadeOut(trajectory),
                run_time=0.3 * run_time_factor
            )
        
        # Add analyzer to scene
        self.add(analyzer)
        
        # Create multiple scattering events
        start_positions = np.linspace(0, 0.5, len(scatter_angles))
        for pos_y, angle, run_time in zip(start_positions, scatter_angles, run_times):
            create_scattering_event(pos_y, angle, run_time)
        
        # Create cone using Surface
        def param_surface(u, v):
            r = u  # radius increases with height
            theta = v
            x = 2 * u  # x goes from 0 to 2
            y = r * np.cos(theta)
            z = r * np.sin(theta)
            return np.array([x, y, z])

        cone = Surface(
            lambda u, v: param_surface(u, v),
            u_range=[0, 1,5],
            v_range=[0, 2*PI],
            resolution=(15, 32),
            fill_opacity=0.3,
            fill_color=BLUE,
            stroke_width=0
        )
        
        # Add incoming line
        incoming_line = DashedLine([-3, 0, 0], [0, 0, 0], color=GREY_C)
        
        # Show final distribution
        self.play(
            Create(incoming_line),
            Create(cone),
            run_time=2
        )
        
        # Stop the rotation and hold
        self.stop_ambient_camera_rotation()
        self.wait()

Manim Community v0.18.1

In [29]:
%%manim -qm NucleonScattering

from manim import *
import numpy as np

class NucleonScattering(ThreeDScene):
    def construct(self):
        # Set up the camera angle
        self.set_camera_orientation(phi=75 * DEGREES, theta=-50 * DEGREES)
        
        # Begin ambient rotation
        self.begin_ambient_camera_rotation(rate=0.2)
        
        # Create the steel analyzer (3D prism)
        analyzer = Prism(
            dimensions=[0.5, 2, 2],
            fill_color=GREY,
            fill_opacity=0.3,
            stroke_width=1
        )
        
        # Define different scattering angles and y-deflections (in degrees)
        scatter_angles = [14, 27, -20, 18, -25, 22]
        y_angles = [5, -8, 10, -5, 7, -10]  # Adding y-direction scattering angles
        run_times = [1.5, 1.2, 1.0, 0.8, 0.6, 0.5]
        
        # Function to create a scattering event
        def create_scattering_event(start_y, angle_deg, y_angle_deg, run_time_factor):
            nucleon = Sphere(radius=0.1, fill_opacity=0.8).set_color(BLUE)
            nucleon.move_to([-3, start_y, 0])
            
            # Calculate end point based on both angles
            xz_angle_rad = angle_deg * DEGREES
            y_angle_rad = y_angle_deg * DEGREES
            end_x = 2
            # Calculate displacement in both y and z
            path_length = np.sqrt(end_x * end_x + start_y * start_y)
            end_y = start_y + path_length * np.tan(xz_angle_rad)
            end_z = path_length * np.tan(y_angle_rad)
            
            path1 = Line([-3, start_y, 0], [0, start_y, 0])
            path2 = Line([0, start_y, 0], [end_x, end_y, end_z])
            
            # Add nucleon
            self.add(nucleon)
            
            # Animate movement
            self.play(
                MoveAlongPath(nucleon, path1),
                rate_func=linear,
                run_time=2 * run_time_factor
            )
            
            self.wait(0.1 * run_time_factor)
            
            self.play(
                MoveAlongPath(nucleon, path2),
                rate_func=linear,
                run_time=1.5 * run_time_factor
            )
            
            # Draw trajectory
            trajectory = VGroup(
                DashedLine([-3, start_y, 0], [0, start_y, 0]),
                DashedLine([0, start_y, 0], [end_x, end_y, end_z])
            ).set_color(GREY_C)
            
            self.play(Create(trajectory), run_time=0.5 * run_time_factor)
            
            # Show angle (projecting onto xz plane for angle measurement)
            ref_line = DashedLine([0, start_y, 0], [2, start_y, 0], color=RED_A)

            # Create angle visualization
            if angle_deg > 0:
                angle_arc = Angle(
                    Line([0, start_y, 0], [2, start_y, 0]),
                    Line([0, start_y, 0], [end_x, end_y, 0]),  # Projected onto xz plane for angle
                    radius=0.5,
                    color=YELLOW,
                    stroke_width=2,
                )
            else:
                angle_arc = Angle(
                    Line([0, start_y, 0], [2, start_y, 0]),
                    Line([0, start_y, 0], [end_x, end_y, 0]),  # Projected onto xz plane for angle
                    radius=0.5,
                    color=YELLOW,
                    stroke_width=2,
                    other_angle=True
                )
            
            angle_label = MathTex(f"{abs(angle_deg)}^{{\circ}}").scale(0.5)
            angle_label.next_to(angle_arc, RIGHT)
            
            self.play(
                Create(ref_line),
                Create(angle_arc),
                Write(angle_label),
                run_time=0.5 * run_time_factor
            )
            
            self.wait(0.3 * run_time_factor)
            
            # Remove everything
            self.play(
                FadeOut(nucleon),
                FadeOut(ref_line),
                FadeOut(angle_arc),
                FadeOut(angle_label),
                FadeOut(trajectory),
                run_time=0.3 * run_time_factor
            )
        
        # Add analyzer to scene
        self.add(analyzer)
        
        # Create multiple scattering events
        start_positions = np.linspace(0, 0.5, len(scatter_angles))
        for pos_y, angle, y_angle, run_time in zip(start_positions, scatter_angles, y_angles, run_times):
            create_scattering_event(pos_y, angle, y_angle, run_time)
        
        # Create cone using Surface
        def param_surface(u, v):
            r = u  # radius increases with height
            theta = v
            x = 2 * u  # x goes from 0 to 2
            y = r * np.cos(theta)
            z = r * np.sin(theta)
            return np.array([x, y, z])

        cone = Surface(
            lambda u, v: param_surface(u, v),
            u_range=[0, 2],
            v_range=[0, 2*PI],
            resolution=(15, 32),
            fill_opacity=0.3,
            fill_color=BLUE,
            stroke_width=0
        )
        
        # Add incoming line
        incoming_line = DashedLine([-3, 0, 0], [0, 0, 0], color=GREY_C)
        
        # Show final distribution
        self.play(
            Create(incoming_line),
            Create(cone),
            run_time=2
        )
        
        # Stop the rotation and hold
        self.stop_ambient_camera_rotation()
        self.wait()

Manim Community v0.18.1

In [30]:
%%manim -qm NucleonScatteringVersions

from manim import *
import numpy as np

class NucleonScatteringVersions(ThreeDScene):
    def construct(self):
        # Common setup
        self.set_camera_orientation(phi=75 * DEGREES, theta=-50 * DEGREES)
        self.begin_ambient_camera_rotation(rate=0.2)
        
        # Create the steel analyzer (3D prism)
        analyzer = Prism(
            dimensions=[0.5, 2, 2],
            fill_color=GREY,
            fill_opacity=0.3,
            stroke_width=1
        )
        self.add(analyzer)
        
        # Version 1: Original smooth cone
        def create_cone():
            def param_surface(u, v):
                r = u
                theta = v
                x = 2 * u
                y = r * np.cos(theta)
                z = r * np.sin(theta)
                return np.array([x, y, z])

            return Surface(
                lambda u, v: param_surface(u, v),
                u_range=[0, 0.8],
                v_range=[0, 2*PI],
                resolution=(15, 32),
                fill_opacity=0.3,
                fill_color=BLUE,
                stroke_width=0
            )
        
        # Version 2: Fan of trajectories
        def create_fan():
            fan = VGroup()
            n_lines = 16
            for i in range(n_lines):
                phi = i * 2 * PI / n_lines
                angle = 20 * DEGREES
                end_radius = 2 * np.tan(angle)
                line = Line(
                    [0, 0, 0],
                    [2, end_radius * np.cos(phi), end_radius * np.sin(phi)],
                    stroke_width=2,
                    color=BLUE_A
                ).set_opacity(0.4)
                fan.add(line)
            return fan
        
        # Version 3: Probability cloud
        def create_cloud():
            cloud = VGroup()
            n_points = 200
            for _ in range(n_points):
                phi = np.random.uniform(0, 2*PI)
                r = np.random.normal(0.4, 0.2)  # Gaussian distribution of radii
                x = np.random.uniform(0, 2)
                sphere = Dot3D(
                    point=[x, r * np.cos(phi), r * np.sin(phi)],
                    radius=0.02,
                    color=BLUE
                ).set_opacity(0.3)
                cloud.add(sphere)
            return cloud

        # Add incoming line
        incoming_line = DashedLine([-3, 0, 0], [0, 0, 0], color=GREY_C)
        
        # Show Version 1: Smooth Cone
        cone = create_cone()
        self.play(
            Create(incoming_line),
            Create(cone),
            run_time=2
        )
        self.wait(2)
        self.play(FadeOut(cone))
        
        # Show Version 2: Fan
        fan = create_fan()
        self.play(
            Create(fan),
            run_time=2
        )
        self.wait(2)
        self.play(FadeOut(fan))
        
        # Show Version 3: Cloud
        cloud = create_cloud()
        self.play(
            Create(cloud),
            run_time=2
        )
        self.wait(2)
        
        # Stop rotation and hold final frame
        self.stop_ambient_camera_rotation()
        self.wait()

Manim Community v0.18.1

In [31]:
%%manim -qm NucleonScatteringVersions

from manim import *
import numpy as np

class NucleonScatteringVersions(ThreeDScene):
    def construct(self):
        # Common setup
        self.set_camera_orientation(phi=75 * DEGREES, theta=-50 * DEGREES)
        self.begin_ambient_camera_rotation(rate=0.2)
        
        # Create the steel analyzer (3D prism)
        analyzer = Prism(
            dimensions=[0.5, 2, 2],
            fill_color=GREY,
            fill_opacity=0.3,
            stroke_width=1
        )
        self.add(analyzer)
        
        # Create reference cone
        def create_cone():
            def param_surface(u, v):
                r = u
                theta = v
                x = 2 * u
                y = r * np.cos(theta)
                z = r * np.sin(theta)
                return np.array([x, y, z])

            return Surface(
                lambda u, v: param_surface(u, v),
                u_range=[0, 0.9],  # Slightly smaller than max trajectory angle
                v_range=[0, 2 * PI],
                resolution=(16, 32),  # Higher resolution for smoother surface (u, v)
                fill_color=BLUE_E,
                fill_opacity=0.2,
                stroke_width=0,
                should_make_jagged=False  # Ensures smooth shading without patterns
            )


        
        # Enhanced fan with random distances
        def create_enhanced_fan():
            fan = VGroup()
            
            # Create evenly spaced azimuthal trajectories
            n_lines = 45  # Total number of lines
            
            for i in range(n_lines):
                # Random azimuthal angle
                phi = i * 2 * PI / n_lines + np.random.uniform(-0.1, 0.1)
                
                # Random polar angle (determines how far out the line goes)
                angle = np.random.uniform(5, 20) * DEGREES
                
                # Random distance
                distance = np.random.uniform(1.5, 2.5)
                end_radius = distance * np.tan(angle)
                
                line = Line(
                    [0, 0, 0],
                    [distance, end_radius * np.cos(phi), end_radius * np.sin(phi)],
                    stroke_width=2,
                    color=BLUE
                ).set_opacity(0.4)
                fan.add(line)
                
            # Add some additional random trajectories
            for _ in range(15):
                phi = np.random.uniform(0, 2*PI)
                angle = np.random.uniform(3, 18) * DEGREES
                distance = np.random.uniform(1.5, 2.5)
                end_radius = distance * np.tan(angle)
                
                line = Line(
                    [0, 0, 0],
                    [distance, end_radius * np.cos(phi), end_radius * np.sin(phi)],
                    stroke_width=2,
                    color=BLUE_D
                ).set_opacity(0.3)
                fan.add(line)
            
            return fan

        # Add incoming and reference lines
        incoming_line = DashedLine([-3, 0, 0], [0, 0, 0], color=GREY_C)
        reference_line = DashedLine([0, 0, 0], [2.5, 0, 0], color=RED_A)
        self.add(incoming_line)
        
        # Create and show cone
        cone = create_cone()
        
        # Create fan
        fan = create_enhanced_fan()
        
        # Show everything together
        self.play(
            Create(fan, lag_ratio=0),
            Create(reference_line),
            run_time=1.5
        )
        self.wait(1)

        self.play(Create(cone))
        self.wait(3)

        
        # Stop rotation and hold final frame
        self.stop_ambient_camera_rotation()
        self.wait(3)

Manim Community v0.18.1

In [32]:
%%manim -qm NucleonScatteringVersions

from manim import *
import numpy as np

class NucleonScatteringVersions(ThreeDScene):
    def construct(self):
        # Common setup
        self.set_camera_orientation(phi=75 * DEGREES, theta=-50 * DEGREES)
        self.begin_ambient_camera_rotation(rate=0.2)
        
        # Create the steel analyzer (3D prism)
        analyzer = Prism(
            dimensions=[0.5, 2, 2],
            fill_color=GREY,
            fill_opacity=0.3,
            stroke_width=1
        )
        self.add(analyzer)
        
        # Create clean cone
        cone = Cone(
            direction=[-1, 0, 0],  # Along x-axis
            height=2.5,
            base_radius=0.7,    
            color=BLUE
        ).set_opacity(0.5)
        
        # Enhanced fan with random distances
        def create_enhanced_fan():
            fan = VGroup()
            
            # Create evenly spaced azimuthal trajectories
            n_lines = 35  # Total number of lines
            
            for i in range(n_lines):
                # Random azimuthal angle
                phi = i * 2 * PI / n_lines + np.random.uniform(-0.1, 0.1)
                
                # Random polar angle
                angle = np.random.uniform(5, 17) * DEGREES
                
                # Random distance
                distance = np.random.uniform(2.2, 2.5)
                end_radius = distance * np.tan(angle)
                
                line = Line(
                    [0, 0, 0],
                    [distance, end_radius * np.cos(phi), end_radius * np.sin(phi)],
                    stroke_width=2,
                    color=BLUE
                ).set_opacity(0.4)
                fan.add(line)
                
            # Add some additional random trajectories
            for _ in range(15):
                phi = np.random.uniform(0, 2*PI)
                angle = np.random.uniform(3, 18) * DEGREES
                distance = np.random.uniform(1.5, 2.5)
                end_radius = distance * np.tan(angle)
                
                line = Line(
                    [0, 0, 0],
                    [distance, end_radius * np.cos(phi), end_radius * np.sin(phi)],
                    stroke_width=2,
                    color=BLUE_D
                ).set_opacity(0.3)
                fan.add(line)
            
            return fan

        # Add incoming and reference lines
        incoming_line = DashedLine([-3, 0, 0], [0, 0, 0], color=GREY_C)
        reference_line = DashedLine([0, 0, 0], [2.5, 0, 0], color=RED_A)
        self.add(incoming_line)
        
        # Create fan
        fan = create_enhanced_fan()
        
        # Show everything together
        self.play(
            Create(fan, lag_ratio=0.01),
            Create(reference_line),
            run_time=1.5
        )
        self.play(Create(cone))
        self.wait(2)
        
        # Stop rotation and hold final frame
        self.stop_ambient_camera_rotation()
        self.wait()

Manim Community v0.18.1

In [37]:
%%manim -qm NucleonScatteringComplete

from manim import *
import numpy as np

class NucleonScatteringComplete(ThreeDScene):
    def construct(self):
        # Common setup
        self.set_camera_orientation(phi=75 * DEGREES, theta=130 * DEGREES)
        self.begin_ambient_camera_rotation(rate=0.2)
        
        # Create the steel analyzer (3D prism)
        analyzer = Prism(
            dimensions=[0.5, 2, 2],
            fill_color=GREY,
            fill_opacity=0.3,
            stroke_width=1
        )
        self.add(analyzer)
        
        # Function for single scattering event
        def create_scattering_event(start_y, angle_deg, y_angle_deg, run_time_factor):
            nucleon = Sphere(radius=0.1, fill_opacity=0.8).set_color(BLUE)
            nucleon.move_to([-3, start_y, 0])
            
            # Calculate end point based on both angles
            xz_angle_rad = angle_deg * DEGREES
            y_angle_rad = y_angle_deg * DEGREES
            end_x = 2
            path_length = np.sqrt(end_x * end_x + start_y * start_y)
            end_y = start_y + path_length * np.tan(xz_angle_rad)
            end_z = path_length * np.tan(y_angle_rad)
            
            path1 = Line([-3, start_y, 0], [0, start_y, 0])
            path2 = Line([0, start_y, 0], [end_x, end_y, end_z])
            
            # Add reference line
            ref_line = DashedLine([0, start_y, 0], [end_x, start_y, 0], color=RED_A)
            
            # Add nucleon
            self.add(nucleon)
            
            # Animate movement
            self.play(
                MoveAlongPath(nucleon, path1),
                rate_func=linear,
                run_time=2 * run_time_factor
            )
            
            self.wait(0.1 * run_time_factor)
            
            self.play(
                MoveAlongPath(nucleon, path2),
                rate_func=linear,
                run_time=1.5 * run_time_factor
            )
            
            # Draw trajectory and reference line
            trajectory = VGroup(
                DashedLine([-3, start_y, 0], [0, start_y, 0]),
                DashedLine([0, start_y, 0], [end_x, end_y, end_z])
            ).set_color(GREY_C)
            
            self.play(
                Create(trajectory),
                Create(ref_line),
                run_time=0.5 * run_time_factor
            )
            self.wait(0.3 * run_time_factor)
            
            # Remove everything
            self.play(
                FadeOut(nucleon),
                FadeOut(trajectory),
                FadeOut(ref_line),
                run_time=0.3 * run_time_factor
            )

        # [Rest of the code remains the same...]
        def create_enhanced_fan():
            fan = VGroup()
            n_lines = 35
            
            for i in range(n_lines):
                phi = i * 2 * PI / n_lines + np.random.uniform(-0.1, 0.1)
                angle = np.random.uniform(5, 17) * DEGREES
                distance = np.random.uniform(2.2, 2.5)
                end_radius = distance * np.tan(angle)
                
                line = Line(
                    [0, 0, 0],
                    [distance, end_radius * np.cos(phi), end_radius * np.sin(phi)],
                    stroke_width=2,
                    color=BLUE
                ).set_opacity(0.4)
                fan.add(line)
            
            for _ in range(15):
                phi = np.random.uniform(0, 2*PI)
                angle = np.random.uniform(3, 18) * DEGREES
                distance = np.random.uniform(1.5, 2.5)
                end_radius = distance * np.tan(angle)
                
                line = Line(
                    [0, 0, 0],
                    [distance, end_radius * np.cos(phi), end_radius * np.sin(phi)],
                    stroke_width=2,
                    color=BLUE_D
                ).set_opacity(0.3)
                fan.add(line)
            
            return fan

        # Show individual scattering events
        scatter_events = [
            (0, 14, 5, 0.9),
            (0.2, -10, -8, 0.7),
            (0.1, 18, 3, 0.5),
            (0.4, -25, 7, 0.3),
            (0.3, 22, -10, 0.3)
        ]
        
        for start_y, angle, y_angle, time_factor in scatter_events:
            create_scattering_event(start_y, angle, y_angle, time_factor)
        
        # Create and show final distribution
        title = Text("Azimuthal Distribution", font_size=24).to_edge(UP)
        self.add_fixed_in_frame_mobjects(title)
        
        incoming_line = DashedLine([-3, 0, 0], [0, 0, 0], color=GREY_C)
        reference_line = DashedLine([0, 0, 0], [2.5, 0, 0], color=RED_A)
        fan = create_enhanced_fan()
        cone = Cone(
            direction=[-1, 0, 0],
            height=2.5,
            base_radius=0.7,    
            color=BLUE
        ).set_opacity(0.5)
        
        # Show distribution
        self.play(
            Create(incoming_line),
            Create(fan, lag_ratio=0.01),
            Create(reference_line),
            run_time=1.5
        )
        self.play(Create(cone))
        self.wait(2)
        
        # Stop rotation and hold final frame
        self.stop_ambient_camera_rotation()
        self.wait()

Manim Community v0.18.1